# Pythonでゴブレットゴブラーズ(簡易)のゲーム木を作成する

- [ゴブレットゴブラーズ (日本語版)](https://sgrk.blog.fc2.com/blog-entry-3687.html)
- [ソースコード設計スライド](https://docs.google.com/presentation/d/1K8fbRlk24Y_J3M--F-WnYYN0hddL89w3EGN4Zgw-L1g/edit?usp=sharing)

- [卒論概要](https://docs.google.com/document/d/1jEXvVmpUPCaowl3xR1qgyhUFGVHFVzyZjBo9F8OAR7M/edit?usp=sharing)

# 必要な関数群

## stateを定義

set_stateとchoise_stateの２種類がある。  
必要な情報を60bitで表す。  
state = 0b_0_0_0_0000_000000000_000000000000000000000000000000000000_00000000


| 区分 | is_first | is_choise | is_choise_board | choise_piece_type | choise_piece | board | hand |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| bit数 | 1bit | 1bit | 1bit | 4bit | 9bit | 36bit | 8bit |
| 例 | 1 | 1 | 1 | 1000 | 100000000 | 010000000 001000000 000110000 000000000 | 00 00 00 00 |
| 説明 | 手番 | モード | 持ち上げている駒は盤か手か | 持ち上げている駒の種類[rl,rm,bl,bm] | 持ち上げている駒のマス[0,1,2,3,4,5,6,7,8] | 盤面に置いてある駒[rl0, rl1, ..., rm0..., bl0...] | 手駒[rlの個数, rmの...] |

説明の補足 :   
- hand  “00”...その駒は持ってない, “01”...一つ持ってる, “11”...２つ持ってる


In [4]:
# 確認
state = 0b_1_0_0_1000_100000000_100000000000000000000000000000000000_10000000
print(state) # 10進数
print(f'{state:060b}')#bin(プレフィックスなし)

print(type(f'{state:060b}')) #文字列です
print(len(f'{state:060b}')) # bit数の確認
# 1bit取り出す
print(f'{state:060b}'[0])
# 複数のbitを取り出す
print(f'{state:060b}'[1+1+1:1+1+1+4])
# 1bit書き換え
print("変更前 : " + f'{state:060b}')
state = int(f'{state:060b}'[:1] + '1' + f'{state:060b}'[2:], 2)
print("変更後 : " + f'{state:060b}')
# 複数のbitを書き換え
print("変更前 : " + f'{state:060b}')
state = int(f'{state:060b}'[:2] + '1' + f'{state:060b}'[3:52] + '11110000', 2)
print("変更後 : " + f'{state:060b}')

# バラす
is_first = f'{state:060b}'[0]
is_choise = f'{state:060b}'[1]
is_choise_board = f'{state:060b}'[2]
choise_piece_type = f'{state:060b}'[3:3+4]
choise_piece = f'{state:060b}'[7:7+9]
board = f'{state:060b}'[16:16+36]
hand = f'{state:060b}'[52:52+8]
print("バラす  : " + is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand)

653030742061744256
100100010000000010000000000000000000000000000000000010000000
<class 'str'>
60
1
1000
変更前 : 100100010000000010000000000000000000000000000000000010000000
変更後 : 110100010000000010000000000000000000000000000000000010000000
変更前 : 110100010000000010000000000000000000000000000000000010000000
変更後 : 111100010000000010000000000000000000000000000000000011110000
バラす  : 111100010000000010000000000000000000000000000000000011110000


## actionを定義

set_action, choise_actionともに、行動する駒を表す。

必要な情報を14bitで表す。  
action = 0b_0_0000_000000000


| 区分 | is_choise_board | choise_piece_type | choise_place |
| ---- | ---- | ---- | ---- | 
| bit数 | 1bit | 4bit | 9bit | 
| 例 | 1 | 1000 | 100000000 | 
| 説明 | 行動する駒は盤か手か | 行動する駒の種類[rl,rm,bl,bm] | 行動する駒のマス[0,1,2,3,4,5,6,7,8] | 


In [5]:
# 確認
action = 0b_1_1000_100000000
print(action) # 10進数
print(f'{action:014b}')#bin(プレフィックスなし)
print(type(f'{action:014b}')) #文字列です
print(len(f'{action:014b}')) # bit数の確認
# 1bit取り出す
print(f'{action:014b}'[0])
# 複数のbitを取り出す
print(f'{action:014b}'[1:1+4])
# 書き換え
print("変更前 : " + f'{action:014b}')
action = int(f'{action:014b}'[:1] + '0001' + '000000010', 2)
print("変更後 : " + f'{action:014b}')


# バラす
is_choise_board = f'{action:014b}'[0]
choise_piece_type = f'{action:014b}'[1:1+4]
choise_piece = f'{action:014b}'[5:5+9]
print("バラす  : " + is_choise_board + choise_piece_type + choise_piece)

12544
11000100000000
<class 'str'>
14
1
1000
変更前 : 11000100000000
変更後 : 10001000000010
バラす  : 10001000000010


## stateからその合法手一覧であるactionsを作成する関数

合法手をすべて列挙する... actions = [action1, action2, …]  
leagal_actions(state) -> actions: を作成
```
def leagal_actions(state) -> actions:
  actions = []
  もし stateがchoise_stateなら、
    actions = leagal_actions_from_choise_state(choise_state)
  そうでなく stateがset_stateなら、
    actions = leagal_actions_from_set_state(set_state)
  return actions
```
stateごとに内部関数を作成。  
- choise_stateから合法手を作成 : leagal_actions_from_choise_state(choise_state)
- set_stateから合法手を作成 : leagal_actions_from_set_state(set_state)

### choise_stateから合法手を作成

  
leagal_actions_from_choise_state(choise_state) -> set_actions:  
setする駒と場所一覧... set_actions を作成する  
setできる駒と場所とは....
- handは行動できない。あくまで選んだ駒を盤面に配置するという作業だから。
- choiseしている駒と同じ駒しか行動できない。例: rlを選んだならrlしか行動できない。
- choiseした駒以上の大きさの駒があるマスには行動できない。
- choiseした駒が盤面なら、
    - choiseした駒があったマスには行動できない。

In [17]:
def leagal_actions_from_choise_state(choise_state):
  is_choise_board = f'{choise_state:060b}'[2]
  choise_piece_type = f'{choise_state:060b}'[3:3+4]
  choise_piece = f'{choise_state:060b}'[7:7+9]
  board = f'{choise_state:060b}'[16:16+36]  

  # set_actions_bitを作成
  set_actions_bit = f'{0:036b}' # 36bit。すべての駒と位置が行動不能で初期化
  # choiseしている駒と同じ駒は行動できる。
  cpy_f = choise_piece_type.find('1')
  set_actions_bit = set_actions_bit[:9*cpy_f] + '111111111' + set_actions_bit[9*(cpy_f+1):]
  # choiseしている駒がboardの駒なら、そのマスに行動出来ない
  if is_choise_board == '1': 
    cp_f = choise_piece.find('1')
    set_actions_bit = set_actions_bit[:9*cpy_f+cp_f] + '0' + set_actions_bit[9*cpy_f+cp_f+1:]
  # choiseした駒以上の大きさの駒があるマスには行動できない。
  for mass in range(9):
    if board[mass] == '1' or board[mass+18] == "1": # L駒があるか
      set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]
    if cpy_f in [1,3]: #選択してあるこまがMなら
      if board[mass+9] == '1' or board[mass+18+9] == "1": # M駒があるか
        set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]

  # actions_bitをactionsに変換
  set_actions = []
  for i, bit in enumerate(set_actions_bit):
    if bit == '1':
      set_action = int('0b_1_0000_000000000', 2)
      set_action = int(f'{set_action:014b}'[:1+(i//9)] + '1' + f'{set_action:014b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      set_action = int(f'{set_action:014b}'[:1+4+(i%9)] + '1' + f'{set_action:014b}'[1+4+(i%9)+1:], 2) # 駒の位置の反映
      set_actions.append(set_action)

  return set_actions

In [22]:
# 確認

# choiseしている駒と同じ駒は行動できる。

# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_1_0_100000_000000000_000000000_000000000_000000000_000000000_01111111
print("先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
# 先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
state = 0b_1_1_0_010000_000000000_000000000_000000000_000000000_000000000_000000000_000000000_11011111
print("先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')

# 後手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
state = 0b_0_1_0_0001_000000000_000000000_000000000_000000000_000000000_11111101
print("後手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
print()

# choiseしている駒がboardの駒なら、そのマスに行動出来ない

# 先手, マス0の自駒Lを選択時, 行動位置は1~8のハズ(盤面に他の駒なし)
state = 0b_1_1_1_1000_100000000_000000000_000000000_000000000_000000000_01111111
print("先手, マス0の自駒Lを選択時, 置ける駒はL, その位置は1~8のハズ(盤面に他の駒なし)")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
# 先手, マス3の自駒Mを選択時, 行動位置は0~2, 4~8のハズ(盤面に他の駒なし)
state = 0b_1_1_1_0100_000100000_000000000_000000000_000000000_000000000_11011111
print("先手, マス3の自駒Mを選択時, 置ける駒はM, その位置は0~2, 4~8のハズ(盤面に他の駒なし)")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
print()

# choiseした駒以上の大きさの駒があるマスには行動できない。

# 先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, 0, BL, BM, 0], 行動位置は0~2, 4~5, 7~8のハズ
state = 0b_1_1_0_1000_000000000_000100000_000010000_000000100_000000010_00010101
print("先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, 0, BL, BM, 0], 行動位置は0~2, 4~5, 7~8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')
# 先手, 手駒Mを選択時, 盤面には[0, 0, 0, RL, RM, 0, BL, BM, 0], 行動位置は0~2, 5, 8のハズ
state = 0b_1_1_0_0100_000000000_000100000_000010000_000000100_000000010_01000101
print("先手, 手駒Mを選択時, 盤面には[0, 0, 0, RL, RM, RS, BL, BM, BS], 行動位置は0~2, 5, 8のハズ")
for action in leagal_actions_from_choise_state(state):
  print(f'{action:014b}')

先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
11000100000000
11000010000000
11000001000000
11000000100000
11000000010000
11000000001000
11000000000100
11000000000010
11000000000001
先手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
10100100000000
10100010000000
10100001000000
10100000100000
10100000010000
10100000001000
10100000000100
10100000000010
10100000000001
後手, 手駒Mを選択時, 置ける駒はM, その位置は0~8のハズ
10001100000000
10001010000000
10001001000000
10001000100000
10001000010000
10001000001000
10001000000100
10001000000010
10001000000001

先手, マス0の自駒Lを選択時, 置ける駒はL, その位置は1~8のハズ(盤面に他の駒なし)
11000010000000
11000001000000
11000000100000
11000000010000
11000000001000
11000000000100
11000000000010
11000000000001
先手, マス3の自駒Mを選択時, 置ける駒はM, その位置は0~2, 4~8のハズ(盤面に他の駒なし)
10100100000000
10100010000000
10100001000000
10100000010000
10100000001000
10100000000100
10100000000010
10100000000001

先手, 手駒Lを選択時, 盤面には[0, 0, 0, RL, RM, 0, BL, BM, 0], 行動位置は0~2, 4~5, 7~8のハズ
11000100000000
11000010000000
11000001000000
11000000010000
11000000001000
11000000

### set_stateから合法手を作成

leagal_actions_from_set_state(set_state) -> choise_actions:  
choiseできる駒の一覧... choise_actions を作成する  
choiseできる駒とは....  
- 自分の駒しか行動できない

手駒なら,
- 手駒に動かしたい駒が１つもなければ、その駒は行動できない。
- 動かしたい駒以上の大きさの駒が配置されているマスには行動できない。

盤上の駒なら、
- 盤上の一番表面にある駒しか行動できない。
- 動かしたい駒以上の大きさの駒があるマスには行動できない。

In [24]:
def leagal_actions_from_set_state(set_state):
  is_first = f'{set_state:060b}'[0]
  board = f'{set_state:060b}'[16:16+36]
  hand = f'{set_state:060b}'[52:52+8]

  # 盤面の各マスに駒が存在するかどうか
  exist_place = f'{int(board[0:9], 2) | int(board[9:18], 2) | int(board[18:27], 2) | int(board[27:36], 2) :09b}' 
  # 手駒は,
  choise_hand_actions_bit = '1111'
  for piece_type in range(4): # 手駒にその駒が１つもないなら行動不可
    if hand[piece_type*2:piece_type*2+2] == "00": 
      choise_hand_actions_bit = choise_hand_actions_bit[:piece_type] + '0' + choise_hand_actions_bit[piece_type+1:]
  if is_first == "1": # 先手なら、先手の駒しか動かせない
    choise_hand_actions_bit = choise_hand_actions_bit[:2] + '00'
  else: # 後手なら、後手の駒しか動かせない
    choise_hand_actions_bit = '00' + choise_hand_actions_bit[2:]
  # 盤上の駒は、
  exist_large_place = int(board[0:9], 2) | int(board[18:27], 2)
  exist_middle_place = int(board[9:18], 2) | int(board[27:36], 2)
  # 盤上の一番表面にある駒しか行動できない
  choise_board_actions_bit = board[0:9] \
   + f'{int(board[9:18], 2) & ~exist_large_place :09b}' \
   + board[18:27] \
   + f'{int(board[27:36], 2) & ~exist_large_place :09b}'
  if is_first == "1": # 先手なら、先手の駒しか動かせない
    choise_board_actions_bit = choise_board_actions_bit[:18] + '000000000000000000'
  else: # 後手なら、後手の駒しか動かせない
    choise_board_actions_bit = '000000000000000000' + choise_board_actions_bit[18:]

  # choise_hand_actions_bitをchoise_actionsへchoise_actionに変換しながら追加
  choise_actions = []
  for i, bit in enumerate(choise_hand_actions_bit): # choise_hand_actions_bit = '00_00
    if bit == '1':
      choise_action = int('0b_0_0000_000000000', 2)
      choise_action = int(f'{choise_action:014b}'[:1+i] + '1' + f'{choise_action:014b}'[1+i+1:], 2) # 駒の種類の反映
      choise_actions.append(choise_action)

  # choise_board_actions_bitをchoise_actionsへchoise_actionに変換しながら追加
  for i, bit in enumerate(choise_board_actions_bit): # choise_board_actions_bit = '000000000_000000000_000000000_000000000'
    if bit == '1':
      choise_action = int('0b_1_0000_000000000', 2)
      choise_action = int(f'{choise_action:014b}'[:1+(i//9)] + '1' + f'{choise_action:014b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      choise_action = int(f'{choise_action:014b}'[:1+4+(i%9)] + '1' + f'{choise_action:014b}'[1+4+(i%9)+1:], 2) # 駒の位置の反映
      choise_actions.append(choise_action)

  return choise_actions

In [26]:
# 確認

# 手駒
# 先手, 手駒にその駒が１つもないなら行動不可
state = 0b_1_0_0_0000_000000000_000000000_000000000_000000000_000000000_00010000
print("# 先手, 手駒にその駒が１つもないなら行動不可。(行動可能は先手の駒数が1つのM駒のハズ)")
for action in leagal_actions_from_set_state(state):
  print(f'{action:014b}')
print()

# 盤上の駒
# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0,1マスにあるL駒と2マスにあるM駒)
state = 0b_1_0_0_0000_000000000_110000000_011000000_000000000_000000000_00001111
print("# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0マスにあるL駒と1マスにあるM駒)")
for action in leagal_actions_from_set_state(state):
  print(f'{action:014b}')

# 先手, 手駒にその駒が１つもないなら行動不可。(行動可能は先手の駒数が1つのM駒のハズ)
00100000000000

# 先手、盤上の一番表面にある駒しか行動できない。(行動可能は0マスにあるL駒と1マスにあるM駒)
11000100000000
11000010000000
10100001000000


### 合法手をすべて列挙する

In [27]:
def legal_actions(state):
  actions = []
  if f'{state:060b}'[1] == '1':
    actions = leagal_actions_from_choise_state(state)
  else:
    actions = leagal_actions_from_set_state(state)
  return actions

In [28]:
# 確認

# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_1_0_1000_000000000_000000000_000000000_000000000_000000000_01111111
print("先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ")
for action in legal_actions(state):
  print(f'{action:014b}')

先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
11000100000000
11000010000000
11000001000000
11000000100000
11000000010000
11000000001000
11000000000100
11000000000010
11000000000001


## state, actionから次のstateを生成する関数の作成


stateにactionを反映して、stateを作成する
```
def create_next_state(state, action) -> next_state:
	next_state = “”
	もし stateがchoise_stateなら、
		next_state = next_state_from_choise_state(choise_state, set_action)
	そうでなく、set_stateなら
		next_state = next_state_from_set_state(set_state, choise_action)
	return next_state
```
stateごとに関数を作成。  
- choise_stateから次のstateを作成 : next_state_from_choise_state(choise_state, set_action)
- set_stateから次のstateを作成 : next_state_from_set_state(set_state, choise_action)


### choise_stateから次のstateを作成

In [29]:
def next_state_from_choise_state(choise_state, set_action):   
  is_first = f'{choise_state:060b}'[0]
  is_choise_board = f'{choise_state:060b}'[2]
  choise_piece_type = f'{choise_state:060b}'[3:3+4]
  choise_piece = f'{choise_state:060b}'[7:7+9]
  board = f'{choise_state:060b}'[16:16+36]
  hand = f'{choise_state:060b}'[52:52+8]

	# set_actionで選択している駒と位置を、boardで1にする。
  piece_type = f'{set_action:014b}'[1:5].find('1')
  piece_place = f'{set_action:014b}'[5:].find('1')
  board = board[:piece_type*9+piece_place] + '1' +  board[piece_type*9+piece_place+1:] 

	# 手番プレイヤーの交代 … is_firstを反転
  is_first = "0" if is_first == "1" else "1"
	# stateのモードを切り替える...is_choiseを0にする
  is_choise = "0"
  # is_choise_board, choise_piece_type, choise_piece をすべて0埋め
  is_choise_board = "0"
  choise_piece_type = "0000"
  choise_piece = "000000000"

  # next_stateの作成
  next_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return next_state

In [32]:
# 確認

# (state = 先手初期浮き手駒Lのchoise盤面, action = Lを0マスに置く) => (state = 後手、先手L駒が0に配置された盤面のset状態)
state = 0b_1_1_0_1000_000000000_000000000000000000000000000000000000_01111111
action = 0b_1_1000_100000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 000000000000000010000000000000000000000000000000000001111111")
print( f'結果 : next_state = {next_state:060b}')
print()

# (state = 後手, 先手Lが0かつ浮き手駒Lのchoise盤面, action = Lを1マスに置く) => (state = 先手、先手L駒が0, 後手L駒が1に配置された盤面のset状態)
state = 0b_0_1_0_0010_000000000_100000000000000000000000000000000000_01110111
action = 0b_1_0010_010000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 100000000000000010000000000000000001000000000000000001110111")
print( f'結果 : next_state = {next_state:060b}')
print()

# (state = 先手, 先手L0後手L1浮き盤駒Lのchoise盤面, action = Lを2マスに置く) => (state = 後手、先手L駒が2、後手L駒が1に配置された盤面のset状態)
state = 0b_1_1_1_1000_100000000_000000000000000000010000000000000000_01110111
action = 0b_1_1000_001000000
next_state = next_state_from_choise_state(state, action)
print("正解 : next_state = 000000000000000000100000000000000001000000000000000001110111")
print( f'結果 : next_state = {next_state:060b}')
print()

正解 : next_state = 000000000000000010000000000000000000000000000000000001111111
結果 : next_state = 000000000000000010000000000000000000000000000000000001111111

正解 : next_state = 100000000000000010000000000000000001000000000000000001110111
結果 : next_state = 100000000000000010000000000000000001000000000000000001110111

正解 : next_state = 000000000000000000100000000000000001000000000000000001110111
結果 : next_state = 000000000000000000100000000000000001000000000000000001110111



### set_stateから次のstateを作成

In [33]:
def next_state_from_set_state(set_state, choise_action):
  is_first = f'{set_state:060b}'[0]
  is_choise_board = f'{set_state:060b}'[2]
  choise_piece_type = f'{set_state:060b}'[3:3+4]
  choise_piece = f'{set_state:060b}'[7:7+9]
  board = f'{set_state:060b}'[16:16+36]
  hand = f'{set_state:060b}'[52:52+8]

	# stateのモードを切り替える...is_choiseを1にする
  is_choise = "1"
  # choise_actionをis_choise_board, choise_piece_type, choise_piece に反映する
  is_choise_board = f'{choise_action:014b}'[0]
  choise_piece_type = f'{choise_action:014b}'[1:5]
  choise_piece =  f'{choise_action:014b}'[5:14]
  # 選択した駒をboard, handから取り除く
  piece_type = f'{choise_action:014b}'[1:5].find('1')
  piece_place = f'{choise_action:014b}'[5:].find('1')
  if f'{choise_action:014b}'[0] == "1":   # choise_actionで盤の駒を選択しているなら
    # choise_actionで選択している駒と位置を、boardで0にする。
    board = board[:piece_type*9+piece_place] + '0' +  board[piece_type*9+piece_place+1:] 
  # choise_actionで選択している駒の数を、handで1減らす。
  # “00”...その駒は持ってない, “01”...一つ持ってる, “11”...２つ持ってる	
  if hand[piece_type*2:piece_type*2+2] == "11":
    hand = hand[:piece_type*2] + "01" + hand[piece_type*2+2:]
  else: # hand[piece_type*2:piece_type*2+2] == "01":
    hand = hand[:piece_type*2] + "00" + hand[piece_type*2+2:]

  # next_stateの作成
  next_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return next_state

### 正規化

In [34]:
def convert_normalization_state(state):
  is_first = f'{state:060b}'[0]
  is_choise = f'{state:060b}'[1]
  is_choise_board = f'{state:060b}'[2]
  choise_piece_type = f'{state:060b}'[3:3+4]
  choise_piece = f'{state:060b}'[7:7+9]
  board = f'{state:060b}'[16:16+36]
  hand = f'{state:060b}'[52:52+8]

  # 変換表を定義
  convert_tables = [[6,3,0,7,4,1,8,5,2],[8,7,6,5,4,3,2,1,0],
                    [2,5,8,1,4,7,0,3,6],[2,1,0,5,4,3,8,7,6],
                    [6,7,8,3,4,5,0,1,2],[0,3,6,1,4,7,2,5,8],[8,5,2,7,4,1,6,3,0]]

  # 変換表でboardを変換したcand_boardを作成し、boardと比較して小さいなら上書きする
  # もしより小さい候補が見つかったら、それと同じ変換表でchoise_pieceも変換する
  board_list = list(board)
  choise_piece_list = list(choise_piece)
  for convert_table in convert_tables:
    cand_board_list = [0] * 36
    for i in range(4):
      for after_num, before_num in enumerate(convert_table):
        cand_board_list[i*9 + after_num] = board_list[i*9 + before_num]
    # print("cand_board : " + "".join(cand_board_list))  # デバッグ用
    # board_list と cand_board_list を比較
    if board_list > cand_board_list:
      board_list = cand_board_list # 上書き
      for after_num, before_num in enumerate(convert_table): # choise_pieceの変換
        choise_piece_list[after_num] = choise_piece_list[before_num]

  # board_list, choise_piece_listをboard, choise_pieceに変換
  board = "".join(board_list)
  choise_piece = "".join(choise_piece_list)

  normalization_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return normalization_state

In [35]:
# 確認
# 先手, 手駒Lを選択時, 置ける駒はL, その位置は0~8のハズ
state = 0b_1_0_0_0000_000000000_100000000_000000000_000000000_000000000_01111111
print("先手L駒0マスにおく -> 先手L駒は8マスに")
print(f'結果 : normalization_state = {convert_normalization_state(state):060b}')

先手L駒0マスにおく -> 先手L駒は8マスに
結果 : normalization_state = 100000000000000000000000100000000000000000000000000001111111


### stateのモードに関わらず、次のstateを生成する

In [36]:
def create_next_state(state, action):
  is_choise = f'{state:060b}'[1]
  next_state = 0
  if is_choise == "1":
    next_state = next_state_from_choise_state(state, action)
  else:
    next_state = next_state_from_set_state(state, action)
  next_state = convert_normalization_state(next_state) # 下記確認ではコメントアウト
  return next_state

In [37]:
# # 確認
# # ※ 正規化前の確認。正規化したら正解と結果は異なる。
# # (state = 先手初期set盤面, action = 手駒Lを選択) => (state = 先手、初期choise盤面、手駒L選択の状態)
# state = 0b_1_0_0_000000_000000000_000000000000000000000000000000000000000000000000000000_111111111111
# action = 0b_0_100000_000000000
# next_state = create_next_state(state, action)
# print("正解 : next_state = 110100000000000000000000000000000000000000000000000000000000000000000000011111111111")
# print( f'結果 : next_state = {next_state:084b}')
# print()

# # (state = 先手初期浮き手駒Lのchoise盤面, action = Lを0マスに置く) => (state = 後手、先手L駒が0に配置された盤面のset状態)
# state = 0b_1_1_0_100000_000000000_000000000000000000000000000000000000000000000000000000_011111111111
# action = 0b_1_100000_100000000
# next_state = create_next_state(state, action)
# print("正解 : next_state = 000000000000000000100000000000000000000000000000000000000000000000000000011111111111")
# print( f'結果 : next_state = {next_state:084b}')
# print()

# # (state = 後手0マスに先手L駒のset盤面, action = 手駒Mを選択) => (state = 後手、マスに先手L駒の盤面、手駒M選択のchoise状態)
# state = 0b_0_0_0_000000_000000000_100000000000000000000000000000000000000000000000000000_011111111111
# action = 0b_0_000010_000000000
# next_state = create_next_state(state, action)
# print("正解 : next_state = 010000010000000000100000000000000000000000000000000000000000000000000000011111110111")
# print( f'結果 : next_state = {next_state:084b}')
# print()

# # (state = 後手先手Lが0かつ浮き手駒Lのchoise盤面, action = Lを1マスに置く) => (state = 先手、先手L駒が0, 後手L駒が1に配置された盤面のset状態)
# state = 0b_0_1_0_000100_000000000_100000000000000000000000000000000000000000000000000000_011111011111
# action = 0b_1_000100_010000000
# next_state = create_next_state(state, action)
# print("正解 : next_state = 100000000000000000100000000000000000000000000010000000000000000000000000011111011111")
# print( f'結果 : next_state = {next_state:084b}')
# print()

# # (state = 先手先手L0後手L1のset盤面, action = 盤駒Lを選択) => (state = 先手、後手L1浮き盤駒Lのchoise盤面、手駒L選択の状態)
# state = 0b_1_0_0_000000_000000000_100000000000000000000000000010000000000000000000000000_011111011111
# action = 0b_1_100000_100000000
# next_state = create_next_state(state, action)
# print("正解 : next_state = 111100000100000000000000000000000000000000000010000000000000000000000000011111011111")
# print( f'結果 : next_state = {next_state:084b}')
# print()

# # (state = 先手先手L0後手L1浮き盤駒Lのchoise盤面, action = Lを2マスに置く) => (state = 後手、先手L駒が2、後手L駒が1に配置された盤面のset状態)
# state = 0b_1_1_1_100000_100000000_000000000000000000000000000010000000000000000000000000_011111011111
# action = 0b_1_100000_001000000
# next_state = create_next_state(state, action)
# print("正解 : next_state = 000000000000000000001000000000000000000000000010000000000000000000000000011111011111")
# print( f'結果 : next_state = {next_state:084b}')
# print()

## 結合テスト



actions が空になるまで、初期盤面からnext_stateを探索していく。  
ここでは、`action = actions[0]`とする。

In [38]:
# 確認
state = 0b_1_0_0_0000_000000000_000000000000000000000000000000000000_11111111
print("0番目")
print("状態 : {}".format(state))
action = legal_actions(state)[0]
print("行動 : {}".format(action))
for i in range(1,10):
  print(str(i) + "番目")
  state = create_next_state(state, action)
  print("状態 : {}".format(state))
  action = legal_actions(state)[0]
  print("行動 : {}".format(action))
  print()


0番目
状態 : 576460752303423743
行動 : 4096
1番目
状態 : 936748722493063295
行動 : 12544

2番目
状態 : 34359738495
行動 : 1024

3番目
状態 : 306244809020932215
行動 : 9472

4番目
状態 : 576460786696716407
行動 : 4096

5番目
状態 : 936748756886356023
行動 : 12416

6番目
状態 : 412325249079
行動 : 1024

7番目
状態 : 306245186986442803
行動 : 9472

8番目
状態 : 576461164662226995
行動 : 2048

9番目
状態 : 900720337832902675
行動 : 10368



## 盤面の判定をする関数の生成 

盤面に対して、lineがあるかどうか、あるならどちらの駒かを確認する。
```
def check_line(state) -> [is_line, is_first_line]:
	stateのboard部分からsurface_boardを作成
	# プレイヤーごとに３目並べの判定をし、もし揃っていたら
		is_line = True
		is_first_line = そのlineがfirst_playerのものかどうか
	# そろってなかったら, is_lineはFalse
	return [is_line, is_first_line]
```

surface_board
 … boardで上に駒が重なっている駒を0にした。

追記: check_result(state) -> [is_done, winner]:に変更  
- is_done : lineがあるか
- winner : 先手なら0, 後手なら1

In [39]:
# 勝敗の有無
def is_win(single_surface):
  s = single_surface
  # 横, 縦, 左斜め, 右斜めのラインを調べる
  check_lines = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]]
  for check_line in check_lines:
    if s[check_line[0]] and s[check_line[1]] and s[check_line[2]]:
      return True;
  return False;
# 表面の駒だけbitが立つboardを作成する
def create_surface(board):
  # boardの変換
  board_list_list = [0] * 4
  for i in range(4):
    board_list_list[i] = list(board[i*9:i*9+9])
  board_surface = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                   [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
  # M駒を反映させる
  board_surface[1] = board_list_list[1]
  board_surface[3] = board_list_list[3]
  # このとき、おいた場所により小さい駒があったら、その駒を0にする
  # １マスずつ見ていく
  for place in range(9):
    # L駒を反映させる
    if board_list_list[0][place] == "1" or board_list_list[2][place] == "1":
      if board_list_list[0][place] == "1":
        board_surface[0][place] = 1
      if board_list_list[2][place] == "1":
        board_surface[2][place] = 1
      board_surface[1][place] = 0 # M駒を0にする
      board_surface[3][place] = 0
  return board_surface

#勝敗の有無、勝者を確認する
def check_result(state):
  board = f'{state:060b}'[16:16+36]
  board_surface = create_surface(board)
  is_done = 0 # 決着がついているなら1を返す
  winner = 0 # 先手は0, 後手は1
  # 内部でboard_surface[0,1]とboard_surface[2,3]を合成
  single_surfaces = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]];
  for i in range(9):
    if board_surface[0][i] == 1 or board_surface[1][i] == 1:
      single_surfaces[0][i] = 1
    elif board_surface[2][i] == 1 or board_surface[3][i] == 1:
      single_surfaces[1][i] = 1
  is_done = is_win(single_surfaces[0]) == 1 or is_win(single_surfaces[1]) == 1
  winner = 1 if is_win(single_surfaces[1]) == 1 else 0
  return [is_done, winner]

# ゲーム木の作成

## 説明

ノードの３状態:
  - 未発見
    unsolved, solvedのいずれにもnext_nodeが存在しない
  - 未訪問
    unsolvedに存在する
  - 既訪問
    solvedに存在する

処理:
- 初期状態"____"を未訪問にする.
- 未訪問キューが空になるまで以下を行う # BFS開始
  1. 未訪問キューから先頭のノードをcurrent_nodeとしてpopする.
  1. 盤面の勝敗判定を行う。
    1. もし、勝敗がついている盤面なら、
      1. RESULTに勝敗を記録
    1. 勝敗がついていない盤面なら
      1. current_nodeを探索し、次のノード(next_nodes)全てをnext_nodesとして列挙する.
      1. next_nodesの要素next_nodeそれぞれに対して、以下を行う.
        1. もし、next_nodeが未発見ならば、
          1. そのノードを未訪問にする.
        1. そうではなく、発見済みならば
          1. 特になし.
        1. next_node.previous_nodeにcurrent_nodeを追加する.
        1. current_node.next_node.にnext_nodeを追加する.
  1. ノード（current_node）を既訪問にする.

1. solvedをcsvで書き出し.

## 実装

### 準備

In [40]:
# # ドライブのマウント
# from google.colab import drive
# drive.mount('/content/drive')

In [41]:
# csvの読み込むためのモジュール
import pandas as pd
from pandas import DataFrame
import numpy as np
# from tabulate import tabulate # pandasのdfをきれいに出力するためのモジュール

### 探索

In [ ]:
%%time 
import time

# 計測
### BFSでゲーム木を作成するプログラム ### 

# state を入力すると、次のstateのリストを出力する関数

def nextStates(state: str) -> list: 
  return next_states

### 設定ここから ###
printFlag = False
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
import warnings
warnings.simplefilter('ignore')
### 設定ここまで ###

###mainここから

# unsolvedDf, solvedDfの初期化
if printFlag:
  print("===")
  print("プログラム開始")
  print("===")
  print()
  print("データを初期化します")
cols = ["PREVIOUS_STATES", "STATE", "NEXT_STATES", "RESULT"] #[前の状態list， 状態, 次の状態list]
df = pd.DataFrame(index=[], columns=cols)
df.set_index("STATE")
unsolvedDf = df
solvedDf = df
if printFlag:
  print("データを初期化しました")
  print()

# 初期状態"____"をunsolvedに追加する。unsolvedに積まれているノードは未訪問.
if printFlag:
  print("===")
  print("BFSの準備")
  print("===")
  print()
  print("初期状態をセットします")
init_state = 0b_1_0_0_0000_000000000_000000000000000000000000000000000000_11111111
previous_state = ""
unsolvedDf = unsolvedDf.append(pd.Series([[previous_state], init_state, "unsolved", ""], index=df.columns, name=init_state))
if printFlag:
  print("初期状態をセットしました") # 確認
  print("確認[UNSOLVED_DF]:") # 確認
  print(unsolvedDf) # 確認
  print() # 確認

# unsolvedが空になるまで以下を行う. BFS開始
if printFlag:
  print("===")
  print("BFSを開始します")
  print("===")
  print()
count_num = 1
# for _ in range(10): # while len(unsolvedDf) > 0: # 開発のためにfor文にしている。
while len(unsolvedDf) > 0:
  time.sleep(0.001) # cpuを休ませる。
  # unsolvedDfから先頭のノードをpopする。
  if len(unsolvedDf) <= 0:
    break;
  current_node = unsolvedDf.iloc[0]  # 先頭のノード(current_node)を抽出。
  unsolvedDf.drop(unsolvedDf.index[0], inplace=True)  # 抽出したノードをunsolvedから削除。
  pass;
  # 勝敗の確認
  is_done, winner = check_result(current_node.STATE)
  # # 勝敗確定盤面なら
  if is_done == 1:
    current_node.RESULT = winner
    current_node.NEXT_STATES = []
  else:  # 勝敗確定盤面でないなら
    pass;
    # 先頭のノード(current_node)から次のノード(next_nodes)を探索する。
    next_states = []
    for action in legal_actions(current_node.STATE):
      next_state = create_next_state(current_node.STATE, action)
      next_states.append(next_state)
    next_states = nextStates(current_node.STATE) # 次のノードの探索結果
    current_node.NEXT_STATES = next_states # current_nodeのNEXT_STATESに探索結果を反映
    # 探索した全ての状態について、以下を行う。
    if printFlag:
      print("unsolvedDfからpopされたノード'{}'の探索を行います".format(current_node.STATE))
    if len(next_states) <= 0:
      if printFlag:
        print("    探索結果: このノードは末端です")
    for next_state in next_states:
      # もし、next_nodeが未発見ならば # unsolved, solvedのいずれにもnext_nodeが存在しない
      if (next_state not in unsolvedDf.STATE.values) and (next_state not in solvedDf.STATE.values):
        # 次のノードが自身と同一
        if next_state == current_node.STATE: 
          if printFlag:
            print("探索結果: 自身のノード'{}'と同一です".format(next_state))
          continue;
        else:
          if printFlag:
            print("    探索結果: 未発見のノード'{}'です".format(next_state))
          count_num = count_num + 1
          print("{}: 未発見のノード".format(count_num))
          # T)そのノードを未訪問にする。 # unsolvedに追加
          previous_state = [current_node.STATE]
          next_node = pd.Series([previous_state, next_state, "unsolved", ""], index=df.columns, name=next_state) # next_nodeの作成
          unsolvedDf = unsolvedDf.append(next_node)
      else:  # F)そうではなく、発見済みならば
        if printFlag:
          print("    探索結果: 発見済みのノード'{}'です".format(next_state))
        #これを既に登録されていたノードのprevious_stateに追加する。
        previous_state = [current_node.STATE] 
        if next_state in unsolvedDf.STATE.values: # unsolvedDfに存在
          if printFlag:
            print("        これはunsolvedに存在しています")
          # unsolvedDf[unsolvedDf.STATE.values == next_state])にprevious_stateを追加する
          tmp = unsolvedDf.loc[next_state, "PREVIOUS_STATES"]
          tmp.append(previous_state[0])       
          unsolvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
        elif next_state in solvedDf.STATE.values:# solveDfに存在
          if printFlag:
            print("        これはsolvedに存在しています")
          # solvedDf[solvedDf.STATE.values == next_state])にprevious_stateを追加する
          tmp = solvedDf.loc[next_state, "PREVIOUS_STATES"]
          tmp.append(previous_state[0])       
          solvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
        else: # 何らかの理由で漏れた状態
          print("        エラー")
  # 現在のノード（current_node）をsolvedDfに追加する。solvedDfのノードは既訪問。 
  solvedDf = solvedDf.append(current_node)

if printFlag:
  print()
  print("BFSが終了しました")
  print()

# 結果確認
# print("===")
# print("結果確認")
# print("===")
# print()
# print("確認[unsolvedDf]:")
# print()
# print(tabulate(unsolvedDf, unsolvedDf.columns,tablefmt='github', showindex=True))
# print()
# print("確認[solvedDf]:")
# print()
# print(tabulate(solvedDf, solvedDf.columns,tablefmt='github', showindex=True))
# print()


### mainここまで

2: 未発見のノード
3: 未発見のノード
4: 未発見のノード
5: 未発見のノード
6: 未発見のノード
7: 未発見のノード
8: 未発見のノード
9: 未発見のノード
10: 未発見のノード
11: 未発見のノード
12: 未発見のノード
13: 未発見のノード
14: 未発見のノード
15: 未発見のノード
16: 未発見のノード
17: 未発見のノード
18: 未発見のノード
19: 未発見のノード
20: 未発見のノード
21: 未発見のノード
22: 未発見のノード
23: 未発見のノード
24: 未発見のノード
25: 未発見のノード
26: 未発見のノード
27: 未発見のノード
28: 未発見のノード
29: 未発見のノード
30: 未発見のノード
31: 未発見のノード
32: 未発見のノード
33: 未発見のノード
34: 未発見のノード
35: 未発見のノード
36: 未発見のノード
37: 未発見のノード
38: 未発見のノード
39: 未発見のノード
40: 未発見のノード
41: 未発見のノード
42: 未発見のノード
43: 未発見のノード
44: 未発見のノード
45: 未発見のノード
46: 未発見のノード
47: 未発見のノード
48: 未発見のノード
49: 未発見のノード
50: 未発見のノード
51: 未発見のノード
52: 未発見のノード
53: 未発見のノード
54: 未発見のノード
55: 未発見のノード
56: 未発見のノード
57: 未発見のノード
58: 未発見のノード
59: 未発見のノード
60: 未発見のノード
61: 未発見のノード
62: 未発見のノード
63: 未発見のノード
64: 未発見のノード
65: 未発見のノード
66: 未発見のノード
67: 未発見のノード
68: 未発見のノード
69: 未発見のノード
70: 未発見のノード
71: 未発見のノード
72: 未発見のノード
73: 未発見のノード
74: 未発見のノード
75: 未発見のノード
76: 未発見のノード
77: 未発見のノード
78: 未発見のノード
79: 未発見のノード
80: 未発見のノード
81: 未発見のノード
82: 未発見のノード
83: 未発見のノード
84: 未発見のノード
85: 未発見のノード


672: 未発見のノード
673: 未発見のノード
674: 未発見のノード
675: 未発見のノード
676: 未発見のノード
677: 未発見のノード
678: 未発見のノード
679: 未発見のノード
680: 未発見のノード
681: 未発見のノード
682: 未発見のノード
683: 未発見のノード
684: 未発見のノード
685: 未発見のノード
686: 未発見のノード
687: 未発見のノード
688: 未発見のノード
689: 未発見のノード
690: 未発見のノード
691: 未発見のノード
692: 未発見のノード
693: 未発見のノード
694: 未発見のノード
695: 未発見のノード
696: 未発見のノード
697: 未発見のノード
698: 未発見のノード
699: 未発見のノード
700: 未発見のノード
701: 未発見のノード
702: 未発見のノード
703: 未発見のノード
704: 未発見のノード
705: 未発見のノード
706: 未発見のノード
707: 未発見のノード
708: 未発見のノード
709: 未発見のノード
710: 未発見のノード
711: 未発見のノード
712: 未発見のノード
713: 未発見のノード
714: 未発見のノード
715: 未発見のノード
716: 未発見のノード
717: 未発見のノード
718: 未発見のノード
719: 未発見のノード
720: 未発見のノード
721: 未発見のノード
722: 未発見のノード
723: 未発見のノード
724: 未発見のノード
725: 未発見のノード
726: 未発見のノード
727: 未発見のノード
728: 未発見のノード
729: 未発見のノード
730: 未発見のノード
731: 未発見のノード
732: 未発見のノード
733: 未発見のノード
734: 未発見のノード
735: 未発見のノード
736: 未発見のノード
737: 未発見のノード
738: 未発見のノード
739: 未発見のノード
740: 未発見のノード
741: 未発見のノード
742: 未発見のノード
743: 未発見のノード
744: 未発見のノード
745: 未発見のノード
746: 未発見のノード
747: 未発見のノード
748: 未発見のノード

1293: 未発見のノード
1294: 未発見のノード
1295: 未発見のノード
1296: 未発見のノード
1297: 未発見のノード
1298: 未発見のノード
1299: 未発見のノード
1300: 未発見のノード
1301: 未発見のノード
1302: 未発見のノード
1303: 未発見のノード
1304: 未発見のノード
1305: 未発見のノード
1306: 未発見のノード
1307: 未発見のノード
1308: 未発見のノード
1309: 未発見のノード
1310: 未発見のノード
1311: 未発見のノード
1312: 未発見のノード
1313: 未発見のノード
1314: 未発見のノード
1315: 未発見のノード
1316: 未発見のノード
1317: 未発見のノード
1318: 未発見のノード
1319: 未発見のノード
1320: 未発見のノード
1321: 未発見のノード
1322: 未発見のノード
1323: 未発見のノード
1324: 未発見のノード
1325: 未発見のノード
1326: 未発見のノード
1327: 未発見のノード
1328: 未発見のノード
1329: 未発見のノード
1330: 未発見のノード
1331: 未発見のノード
1332: 未発見のノード
1333: 未発見のノード
1334: 未発見のノード
1335: 未発見のノード
1336: 未発見のノード
1337: 未発見のノード
1338: 未発見のノード
1339: 未発見のノード
1340: 未発見のノード
1341: 未発見のノード
1342: 未発見のノード
1343: 未発見のノード
1344: 未発見のノード
1345: 未発見のノード
1346: 未発見のノード
1347: 未発見のノード
1348: 未発見のノード
1349: 未発見のノード
1350: 未発見のノード
1351: 未発見のノード
1352: 未発見のノード
1353: 未発見のノード
1354: 未発見のノード
1355: 未発見のノード
1356: 未発見のノード
1357: 未発見のノード
1358: 未発見のノード
1359: 未発見のノード
1360: 未発見のノード
1361: 未発見のノード
1362: 未発見のノード
1363: 未発見のノード
1364: 

1911: 未発見のノード
1912: 未発見のノード
1913: 未発見のノード
1914: 未発見のノード
1915: 未発見のノード
1916: 未発見のノード
1917: 未発見のノード
1918: 未発見のノード
1919: 未発見のノード
1920: 未発見のノード
1921: 未発見のノード
1922: 未発見のノード
1923: 未発見のノード
1924: 未発見のノード
1925: 未発見のノード
1926: 未発見のノード
1927: 未発見のノード
1928: 未発見のノード
1929: 未発見のノード
1930: 未発見のノード
1931: 未発見のノード
1932: 未発見のノード
1933: 未発見のノード
1934: 未発見のノード
1935: 未発見のノード
1936: 未発見のノード
1937: 未発見のノード
1938: 未発見のノード
1939: 未発見のノード
1940: 未発見のノード
1941: 未発見のノード
1942: 未発見のノード
1943: 未発見のノード
1944: 未発見のノード
1945: 未発見のノード
1946: 未発見のノード
1947: 未発見のノード
1948: 未発見のノード
1949: 未発見のノード
1950: 未発見のノード
1951: 未発見のノード
1952: 未発見のノード
1953: 未発見のノード
1954: 未発見のノード
1955: 未発見のノード
1956: 未発見のノード
1957: 未発見のノード
1958: 未発見のノード
1959: 未発見のノード
1960: 未発見のノード
1961: 未発見のノード
1962: 未発見のノード
1963: 未発見のノード
1964: 未発見のノード
1965: 未発見のノード
1966: 未発見のノード
1967: 未発見のノード
1968: 未発見のノード
1969: 未発見のノード
1970: 未発見のノード
1971: 未発見のノード
1972: 未発見のノード
1973: 未発見のノード
1974: 未発見のノード
1975: 未発見のノード
1976: 未発見のノード
1977: 未発見のノード
1978: 未発見のノード
1979: 未発見のノード
1980: 未発見のノード
1981: 未発見のノード
1982: 

2502: 未発見のノード
2503: 未発見のノード
2504: 未発見のノード
2505: 未発見のノード
2506: 未発見のノード
2507: 未発見のノード
2508: 未発見のノード
2509: 未発見のノード
2510: 未発見のノード
2511: 未発見のノード
2512: 未発見のノード
2513: 未発見のノード
2514: 未発見のノード
2515: 未発見のノード
2516: 未発見のノード
2517: 未発見のノード
2518: 未発見のノード
2519: 未発見のノード
2520: 未発見のノード
2521: 未発見のノード
2522: 未発見のノード
2523: 未発見のノード
2524: 未発見のノード
2525: 未発見のノード
2526: 未発見のノード
2527: 未発見のノード
2528: 未発見のノード
2529: 未発見のノード
2530: 未発見のノード
2531: 未発見のノード
2532: 未発見のノード
2533: 未発見のノード
2534: 未発見のノード
2535: 未発見のノード
2536: 未発見のノード
2537: 未発見のノード
2538: 未発見のノード
2539: 未発見のノード
2540: 未発見のノード
2541: 未発見のノード
2542: 未発見のノード
2543: 未発見のノード
2544: 未発見のノード
2545: 未発見のノード
2546: 未発見のノード
2547: 未発見のノード
2548: 未発見のノード
2549: 未発見のノード
2550: 未発見のノード
2551: 未発見のノード
2552: 未発見のノード
2553: 未発見のノード
2554: 未発見のノード
2555: 未発見のノード
2556: 未発見のノード
2557: 未発見のノード
2558: 未発見のノード
2559: 未発見のノード
2560: 未発見のノード
2561: 未発見のノード
2562: 未発見のノード
2563: 未発見のノード
2564: 未発見のノード
2565: 未発見のノード
2566: 未発見のノード
2567: 未発見のノード
2568: 未発見のノード
2569: 未発見のノード
2570: 未発見のノード
2571: 未発見のノード
2572: 未発見のノード
2573: 

...

21638: 未発見のノード
21639: 未発見のノード
21640: 未発見のノード
21641: 未発見のノード
21642: 未発見のノード
21643: 未発見のノード
21644: 未発見のノード
21645: 未発見のノード
21646: 未発見のノード
21647: 未発見のノード
21648: 未発見のノード
21649: 未発見のノード
21650: 未発見のノード
21651: 未発見のノード
21652: 未発見のノード
21653: 未発見のノード
21654: 未発見のノード
21655: 未発見のノード
21656: 未発見のノード
21657: 未発見のノード
21658: 未発見のノード
21659: 未発見のノード
21660: 未発見のノード
21661: 未発見のノード
21662: 未発見のノード
21663: 未発見のノード
21664: 未発見のノード
21665: 未発見のノード
21666: 未発見のノード
21667: 未発見のノード
21668: 未発見のノード
21669: 未発見のノード
21670: 未発見のノード
21671: 未発見のノード
21672: 未発見のノード
21673: 未発見のノード
21674: 未発見のノード
21675: 未発見のノード
21676: 未発見のノード
21677: 未発見のノード
21678: 未発見のノード
21679: 未発見のノード
21680: 未発見のノード
21681: 未発見のノード
21682: 未発見のノード
21683: 未発見のノード
21684: 未発見のノード
21685: 未発見のノード
21686: 未発見のノード
21687: 未発見のノード
21688: 未発見のノード
21689: 未発見のノード
21690: 未発見のノード
21691: 未発見のノード
21692: 未発見のノード
21693: 未発見のノード
21694: 未発見のノード
21695: 未発見のノード
21696: 未発見のノード
21697: 未発見のノード
21698: 未発見のノード
21699: 未発見のノード
21700: 未発見のノード
21701: 未発見のノード
21702: 未発見のノード
21703: 未発見のノード
21704: 未発見

22191: 未発見のノード
22192: 未発見のノード
22193: 未発見のノード
22194: 未発見のノード
22195: 未発見のノード
22196: 未発見のノード
22197: 未発見のノード
22198: 未発見のノード
22199: 未発見のノード
22200: 未発見のノード
22201: 未発見のノード
22202: 未発見のノード
22203: 未発見のノード
22204: 未発見のノード
22205: 未発見のノード
22206: 未発見のノード
22207: 未発見のノード
22208: 未発見のノード
22209: 未発見のノード
22210: 未発見のノード
22211: 未発見のノード
22212: 未発見のノード
22213: 未発見のノード
22214: 未発見のノード
22215: 未発見のノード
22216: 未発見のノード
22217: 未発見のノード
22218: 未発見のノード
22219: 未発見のノード
22220: 未発見のノード
22221: 未発見のノード
22222: 未発見のノード
22223: 未発見のノード
22224: 未発見のノード
22225: 未発見のノード
22226: 未発見のノード
22227: 未発見のノード
22228: 未発見のノード
22229: 未発見のノード
22230: 未発見のノード
22231: 未発見のノード
22232: 未発見のノード
22233: 未発見のノード
22234: 未発見のノード
22235: 未発見のノード
22236: 未発見のノード
22237: 未発見のノード
22238: 未発見のノード
22239: 未発見のノード
22240: 未発見のノード
22241: 未発見のノード
22242: 未発見のノード
22243: 未発見のノード
22244: 未発見のノード
22245: 未発見のノード
22246: 未発見のノード
22247: 未発見のノード
22248: 未発見のノード
22249: 未発見のノード
22250: 未発見のノード
22251: 未発見のノード
22252: 未発見のノード
22253: 未発見のノード
22254: 未発見のノード
22255: 未発見のノード
22256: 未発見のノード
22257: 未発見

22748: 未発見のノード
22749: 未発見のノード
22750: 未発見のノード
22751: 未発見のノード
22752: 未発見のノード
22753: 未発見のノード
22754: 未発見のノード
22755: 未発見のノード
22756: 未発見のノード
22757: 未発見のノード
22758: 未発見のノード
22759: 未発見のノード
22760: 未発見のノード
22761: 未発見のノード
22762: 未発見のノード
22763: 未発見のノード
22764: 未発見のノード
22765: 未発見のノード
22766: 未発見のノード
22767: 未発見のノード
22768: 未発見のノード
22769: 未発見のノード
22770: 未発見のノード
22771: 未発見のノード
22772: 未発見のノード
22773: 未発見のノード
22774: 未発見のノード
22775: 未発見のノード
22776: 未発見のノード
22777: 未発見のノード
22778: 未発見のノード
22779: 未発見のノード
22780: 未発見のノード
22781: 未発見のノード
22782: 未発見のノード
22783: 未発見のノード
22784: 未発見のノード
22785: 未発見のノード
22786: 未発見のノード
22787: 未発見のノード
22788: 未発見のノード
22789: 未発見のノード
22790: 未発見のノード
22791: 未発見のノード
22792: 未発見のノード
22793: 未発見のノード
22794: 未発見のノード
22795: 未発見のノード
22796: 未発見のノード
22797: 未発見のノード
22798: 未発見のノード
22799: 未発見のノード
22800: 未発見のノード
22801: 未発見のノード
22802: 未発見のノード
22803: 未発見のノード
22804: 未発見のノード
22805: 未発見のノード
22806: 未発見のノード
22807: 未発見のノード
22808: 未発見のノード
22809: 未発見のノード
22810: 未発見のノード
22811: 未発見のノード
22812: 未発見のノード
22813: 未発見のノード
22814: 未発見

23297: 未発見のノード
23298: 未発見のノード
23299: 未発見のノード
23300: 未発見のノード
23301: 未発見のノード
23302: 未発見のノード
23303: 未発見のノード
23304: 未発見のノード
23305: 未発見のノード
23306: 未発見のノード
23307: 未発見のノード
23308: 未発見のノード
23309: 未発見のノード
23310: 未発見のノード
23311: 未発見のノード
23312: 未発見のノード
23313: 未発見のノード
23314: 未発見のノード
23315: 未発見のノード
23316: 未発見のノード
23317: 未発見のノード
23318: 未発見のノード
23319: 未発見のノード
23320: 未発見のノード
23321: 未発見のノード
23322: 未発見のノード
23323: 未発見のノード
23324: 未発見のノード
23325: 未発見のノード
23326: 未発見のノード
23327: 未発見のノード
23328: 未発見のノード
23329: 未発見のノード
23330: 未発見のノード
23331: 未発見のノード
23332: 未発見のノード
23333: 未発見のノード
23334: 未発見のノード
23335: 未発見のノード
23336: 未発見のノード
23337: 未発見のノード
23338: 未発見のノード
23339: 未発見のノード
23340: 未発見のノード
23341: 未発見のノード
23342: 未発見のノード
23343: 未発見のノード
23344: 未発見のノード
23345: 未発見のノード
23346: 未発見のノード
23347: 未発見のノード
23348: 未発見のノード
23349: 未発見のノード
23350: 未発見のノード
23351: 未発見のノード
23352: 未発見のノード
23353: 未発見のノード
23354: 未発見のノード
23355: 未発見のノード
23356: 未発見のノード
23357: 未発見のノード
23358: 未発見のノード
23359: 未発見のノード
23360: 未発見のノード
23361: 未発見のノード
23362: 未発見のノード
23363: 未発見

23847: 未発見のノード
23848: 未発見のノード
23849: 未発見のノード
23850: 未発見のノード
23851: 未発見のノード
23852: 未発見のノード
23853: 未発見のノード
23854: 未発見のノード
23855: 未発見のノード
23856: 未発見のノード
23857: 未発見のノード
23858: 未発見のノード
23859: 未発見のノード
23860: 未発見のノード
23861: 未発見のノード
23862: 未発見のノード
23863: 未発見のノード
23864: 未発見のノード
23865: 未発見のノード
23866: 未発見のノード
23867: 未発見のノード
23868: 未発見のノード
23869: 未発見のノード
23870: 未発見のノード
23871: 未発見のノード
23872: 未発見のノード
23873: 未発見のノード
23874: 未発見のノード
23875: 未発見のノード
23876: 未発見のノード
23877: 未発見のノード
23878: 未発見のノード
23879: 未発見のノード
23880: 未発見のノード
23881: 未発見のノード
23882: 未発見のノード
23883: 未発見のノード
23884: 未発見のノード
23885: 未発見のノード
23886: 未発見のノード
23887: 未発見のノード
23888: 未発見のノード
23889: 未発見のノード
23890: 未発見のノード
23891: 未発見のノード
23892: 未発見のノード
23893: 未発見のノード
23894: 未発見のノード
23895: 未発見のノード
23896: 未発見のノード
23897: 未発見のノード
23898: 未発見のノード
23899: 未発見のノード
23900: 未発見のノード
23901: 未発見のノード
23902: 未発見のノード
23903: 未発見のノード
23904: 未発見のノード
23905: 未発見のノード
23906: 未発見のノード
23907: 未発見のノード
23908: 未発見のノード
23909: 未発見のノード
23910: 未発見のノード
23911: 未発見のノード
23912: 未発見のノード
23913: 未発見

### 出力

In [46]:
# solvedDfをox_outputという名前で書き出し
solvedDf.to_csv('/Users/toku/Desktop/ox_output.csv')

In [47]:
# ox_outputの確認

solvedDf = pd.read_csv(
    "/Users/toku/Desktop/ox_output.csv",
    index_col=0, # 最初の１行はデータ名。
    encoding="cp932" # windowsの追加文字に対応。おまじないだと思えば良い。
    )
print(solvedDf)

                                                      PREVIOUS_STATES  \
576460752303423743                                               ['']   
936748722493063295                               [576460752303423743]   
900719925474099423                               [576460752303423743]   
34359738495         [936748722493063295, 936748722493063295, 93674...   
68719476863         [936748722493063295, 936748722493063295, 93674...   
549755814015                                     [936748722493063295]   
67109087            [900719925474099423, 900719925474099423, 90071...   
134217951           [900719925474099423, 900719925474099423, 90071...   
1073742047                                       [900719925474099423]   
306244809020932215                                      [34359738495]   

                                 STATE  \
576460752303423743  576460752303423743   
936748722493063295  936748722493063295   
900719925474099423  900719925474099423   
34359738495                3